In [1]:
%matplotlib inline

import numpy as np
import os
import matplotlib.pylab as plt
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.applications.vgg16 import VGG16
from keras import optimizers

In [6]:
# Data preprocessing and augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, zoom_range=0.3, horizontal_flip=True)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

TRAIN_DATA_PATH = '/Users/kaylienguyen/Downloads/FER_dataset/trainrevised'
TEST_DATA_PATH = '/Users/kaylienguyen/Downloads/FER_dataset/testrevised'
IMAGE_SIZE = (48, 48)
BATCH_SIZE = 64

In [7]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    TEST_DATA_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False)

Found 28614 images belonging to 7 classes.
Found 7071 images belonging to 7 classes.


In [10]:
input_shape = (48, 48, 1)
epochs = 10
conv_base = VGG16(weights='imagenet',
                include_top=False,
                input_shape=(48, 48, 3))
conv_base.trainable = False

model = Sequential([
    Conv2D(filters=10, kernel_size=3, strides=(1, 1), padding='valid', activation='relu', input_shape=input_shape, kernel_regularizer=l2(.01)),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    BatchNormalization(),
    Conv2D(filters=20, kernel_size=3, strides=(1, 1), padding='valid', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    BatchNormalization(),
    #Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='valid', activation='relu'),
    #Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='valid', activation='relu'),
    #Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='valid', activation='relu'),
    Flatten(),
    Dense(1024, activation = 'relu'),
    Dense(512, activation = 'relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(7, activation='softmax')
])

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False


In [11]:
from keras.callbacks import ReduceLROnPlateau

initial_learning_rate = 1e-4

# Define a learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',   
    factor=0.2,           # Factor by which the learning rate will be reduced (new_lr = lr * factor)
    patience=3,           # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6           # Lower bound on the learning rate
)


model.compile(
    optimizer=optimizers.Adam(lr=initial_learning_rate),  
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


model.fit(
    train_generator,
    epochs=epochs,
    batch_size=64,
    validation_split=0.2,
    validation_data = test_generator,
    callbacks=[lr_scheduler]  
)

#model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizers.Adam(lr=1e-4))
#model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

test_loss, test_accuracy = model.evaluate_generator(test_generator)
print('Test Loss: ', test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/10
448/448 [==============================] - 26s 57ms/step - loss: 1.7717 - accuracy: 0.3007 - val_loss: 1.7137 - val_accuracy: 0.3007 - lr: 0.0010
Epoch 2/10
448/448 [==============================] - 24s 53ms/step - loss: 1.6011 - accuracy: 0.3794 - val_loss: 1.5203 - val_accuracy: 0.4021 - lr: 0.0010
Epoch 3/10
448/448 [==============================] - 25s 55ms/step - loss: 1.5276 - accuracy: 0.4101 - val_loss: 1.4385 - val_accuracy: 0.4477 - lr: 0.0010
Epoch 4/10
448/448 [==============================] - 25s 55ms/step - loss: 1.4720 - accuracy: 0.4358 - val_loss: 1.4251 - val_accuracy: 0.4533 - lr: 0.0010
Epoch 5/10
448/448 [==============================] - 25s 56ms/step - loss: 1.4268 - accuracy: 0.4560 - val_loss: 1.3534 - val_accuracy: 0.4830 - lr: 0.0010
Epoch 6/10
448/448 [==============================] - 24s 53ms/step - loss: 1.3867 - accuracy: 0.4735 - val_loss: 1.3053 - val_accuracy: 0.4996 - lr: 0.0010
Epoch 7/10
448/448 [==============================] - 24s 

/var/folders/3m/hxyrmk3x0fndxrfylccz8ldh0000gn/T/ipykernel_98249/2991971360.py:37: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy = model.evaluate_generator(test_generator)


Test Loss:  1.3824172019958496
Test Accuracy: 0.4698062539100647
